In [ ]:
!pip install pyproj

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install cython
!pip install cython
!pip uninstall cartopy
!pip install cartopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Found existing installation: Cartopy 0.21.1
Uninstalling Cartopy-0.21.1:
  Would remove:
    /usr/local/bin/cartopy_feature_download.py
    /usr/local/lib/python3.9/dist-packages/Cartopy-0.21.1.dist-info/*
    /usr/local/lib/python3.9/dist-packages/cartopy/*
Proceed (Y/n)? y
  Successfully uninstalled Cartopy-0.21.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached Cartopy-0.21.1-cp39-cp39-linux_x86_64.whl


In [ ]:
!apt-get -qq install python-cartopy python3-cartopy
!pip uninstall -y shapely    
!pip install shapely --no-binary shapely

E: Unable to locate package python-cartopy
Found existing installation: shapely 2.0.1
Uninstalling shapely-2.0.1:
  Successfully uninstalled shapely-2.0.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached shapely-2.0.1.tar.gz (275 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for shapely: filename=shapely-2.0.1-cp39-cp39-linux_x86_64.whl size=979758 sha256=79633330ec64cc76baed7620c6ea367026d350f5f7f95e735a6e4cc430b6f8c5
  Stored in directory: /root/.cache/pip/wheels/c2/f0/2b/e33fb469c9bf8d08688be4df902af6d7c2210128fc7c046889
Successfully built shapely


In [ ]:
import pandas as pd
import pyproj
import math
import matplotlib.pyplot as plt
import seaborn as sb
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import cartopy
import cartopy.crs as ccrs                   
import cartopy.feature as cfeature           
import cartopy.io.shapereader as shapereader
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.feature.nightshade import Nightshade
import geopandas as gpd
import random
from shapely.geometry import Point, Polygon
import time

In [ ]:
root = 'https://raw.githubusercontent.com/fmoreno4/cd-files/main/eventos_dengue_2020.csv'
df = pd.read_csv(root)
df

URLError: ignored

In [ ]:
#Creamos la proyeccion
P = pyproj.Proj(proj='utm', zone=15, ellps='WGS84', preserve_units=True)

In [ ]:
#Convertimos las cordenadas a UTM
df['x'], df['y'] = P(df['longitud'], df['latitud'])
df

In [ ]:
subplot_kw = dict(projection=ccrs.PlateCarree())

fig, ax = plt.subplots(figsize=(16, 16), subplot_kw=subplot_kw)

# Puntos Floridablanca
lower_lon = -73.13
upper_lon = -73.08
lower_lat = 7.06
upper_lat = 7.10

ax.scatter(df['longitud'], df['latitud'], zorder=10, c='b', s=10)
plt.show()

In [ ]:
#Se carga la cartografía de Floridablanca
url = 'https://foscal.net/cartografia/'

file_list = ["68276_urbano.cpg", "68276_urbano.dbf", "68276_urbano.prj", "68276_urbano.qmd", "68276_urbano.shp", "68276_urbano.shx"]

for file in file_list:
    urllib.request.urlretrieve(url+file, file)

In [ ]:
subplot_kw = dict(projection=ccrs.PlateCarree())

fig, ax = plt.subplots(figsize=(20, 20), subplot_kw=subplot_kw)

# Mapa de Floridablanca
lower_lon = -73.13
upper_lon = -73.08
lower_lat = 7.06
upper_lat = 7.10

ax.set_extent([lower_lon, upper_lon, lower_lat, upper_lat])

reader = shapereader.Reader('68276_urbano.shp')

for poly in reader.geometries():
    ax.add_geometries([poly], ccrs.PlateCarree(), facecolor='white', edgecolor='black')

plt.show()

In [ ]:
#Combinacion de las dos graficas
fdbnc = gpd.read_file("68276_urbano.shp")
fdbnc = fdbnc.to_crs("EPSG:32618")
area_fdbnc = fdbnc.area.sum()
print(f"El área de Floridablanca es {area_fdbnc:.2f} metros cuadrados")


In [ ]:
pointList=[]
for i in range(df.shape[0]):
  pointList.append((df["x"][i],df["y"][i]))

In [ ]:
len(pointList)

In [ ]:
from shapely.geometry import Point
from shapely.ops import nearest_points
from itertools import combinations

In [ ]:
from scipy.spatial.distance import pdist, squareform

coordenadas = np.column_stack((df['x'], df['y']))

dist_matrix = squareform(pdist(coordenadas))

mask = dist_matrix < 100

pairNum = np.sum(mask) // 2 
print(pairNum)

In [ ]:
def calcular_distancia(x1, y1, x2, y2):
    distancia = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distancia

In [ ]:
def k_ripley(pointsList,area,h):
  contK=0
  for i in range(len(pointsList)-1):
    for j in range(i+1,len(pointsList)):
      if calcular_distancia(pointsList[i][0],pointsList[i][1],pointsList[j][0],pointsList[j][1])<=h:
        contK+=1
  valorK=contK*area/len(pointsList)**2
  return valorK 
  
k_ripley(pointList,area_fdbnc,100)

In [ ]:
lower_lon, lower_lat, upper_lon, upper_lat = fdbnc.total_bounds
num_pnts = 1000

random_pnts = []
while len(random_pnts) < num_pnts:
    pnt = [np.random.uniform(upper_lon, lower_lon), np.random.uniform(lower_lat, upper_lat)]
    random_pnts.append(pnt)



In [ ]:
def arePointsInShape(polygon, points):
    filtered_pnts = []
    for point in points:
        if polygon.contains(Point(point)):
            filtered_pnts.append(point)
    return filtered_pnts

filtered_pnts = arePointsInShape(fdbnc.geometry[0], random_pnts)

In [ ]:
pnts_gdf = gpd.GeoDataFrame(geometry=[Point(point) for point in filtered_pnts])
pnts_gdf['id'] = range(len(pnts_gdf))

In [ ]:
ax = fdbnc.plot(color='white', edgecolor='black')
pnts_gdf.plot(ax=ax, color='blue', markersize=5)

In [ ]:
p = pyproj.Proj(proj='utm', zone=18, ellps='WGS84', preserve_units=True)

In [ ]:
lower_lon = -73.13
upper_lon = -73.08
lower_lat = 7.06
upper_lat = 7.10

x_min,y_min = p(lower_lon, lower_lat)
x_max, y_max = p(upper_lon, upper_lat)

In [ ]:
x_min,y_min,x_max,y_max

In [ ]:
import shapely.geometry as sg


In [ ]:
from shapely.geometry import Polygon
coords = fdbnc.geometry[0]
polygon = Polygon(coords)
ramList=[]
for i in range(9):
    group = []
    i = 0
while i < len(pointList):
    random_easting = np.random.uniform(x_min, x_max)
    random_northing = np.random.uniform(y_min, y_max)
    point = Point(random_easting, random_northing)
    if point.within(polygon):
        group.append((random_easting, random_northing))
        i += 1

ramList.append(group)

In [ ]:
df1 = pd.DataFrame(ramList[0], columns=['x', 'y'])

df1['longitud'],df1['latitud']=p(df1['x'],df1['y'],inverse=True)

df1

In [ ]:
from pandas.compat import F
dataList=[]
for h in range(10,201,10):
    k = k_ripley(pointList,area_fdbnc,h)
    k_list=[]
    for group in ramList:
        k_list.append(k_ripley(group,area_fdbnc,h))
    k_list.append(k)
    k_list.sort()

    first_percentile = np.percentile(k_list, 2.5)
    second_percentile = np.percentile(k_list, 97.5)
    dataList.append([h, k, first_percentile, second_percentile])

In [ ]:
results=pd.DataFrame(dataList,columns=('h','k','first_percentile', 'second_percentile'))
results


In [ ]:
df = pd.DataFrame(dataList, columns=['h', 'k', 'percentile_2_5', 'percentile_97_5'])

# Creamos la visualización
plt.plot(df['h'], df['k'], label='k')
plt.plot(df['h'], df['percentile_2_5'], label='2.5% percentile')
plt.plot(df['h'], df['percentile_97_5'], label='97.5% percentile')
plt.xlabel('h')
plt.ylabel('Valor')
plt.legend()
plt.show()